In [1]:
import requests
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

%matplotlib inline
plt.style.use('ggplot')

In [3]:
df = pd.read_csv('athlete_events.csv')
df.head()

ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal  
0       Basketball Men's Basketball   NaN  
1      Judo Men's Extra-Lightweight   NaN  
2           Football Men's Football   NaN  
3       Tug-Of-War Men's Tug-Of-War  Gold  
4  Speed Skating Women's 500 metres   NaN

In [4]:


df_OLY_all = df[df['Sport'].str.contains('Basketball', regex=True)]
df_OLY_all.head()

ID                                Name Sex   Age  Height  Weight   Team  \
0      1                           A Dijiang   M  24.0   180.0    80.0  China   
167   69                   Tamara Abalde Daz   F  19.0   185.0    72.0  Spain   
250  124               Youssef Mohamed Abbas   M  31.0     NaN     NaN  Egypt   
264  136                    Alessandro Abbio   M  29.0   195.0    85.0  Italy   
346  192  Ahmed El-Sayed Abdel Hamid Mobarak   M  25.0   189.0    85.0  Egypt   

     NOC        Games  Year  Season       City       Sport  \
0    CHN  1992 Summer  1992  Summer  Barcelona  Basketball   
167  ESP  2008 Summer  2008  Summer    Beijing  Basketball   
250  EGY  1952 Summer  1952  Summer   Helsinki  Basketball   
264  ITA  2000 Summer  2000  Summer     Sydney  Basketball   
346  EGY  1972 Summer  1972  Summer     Munich  Basketball   

                             Event Medal  
0      Basketball Men's Basketball   NaN  
167  Basketball Women's Basketball   NaN  
250    Basketball Men's Basketball   NaN  
264    Basketball Men's Basketball   NaN  
346    Basketball Men's Basketball   NaN

In [5]:
df_OLY = df_OLY_all[df_OLY_all['Team']=='China']
df_OLY.head()

ID           Name Sex   Age  Height  Weight   Team  NOC  \
0          1      A Dijiang   M  24.0   180.0    80.0  China  CHN   
11690   6381         Ba Yan   F  21.0   183.0    78.0  China  CHN   
21295  11239       Bian Lan   F  17.0   180.0    73.0  China  CHN   
21296  11239       Bian Lan   F  21.0   180.0    73.0  China  CHN   
39283  20211  Chen Jianghua   M  19.0   187.0    73.0  China  CHN   

             Games  Year  Season         City       Sport  \
0      1992 Summer  1992  Summer    Barcelona  Basketball   
11690  1984 Summer  1984  Summer  Los Angeles  Basketball   
21295  2004 Summer  2004  Summer       Athina  Basketball   
21296  2008 Summer  2008  Summer      Beijing  Basketball   
39283  2008 Summer  2008  Summer      Beijing  Basketball   

                               Event   Medal  
0        Basketball Men's Basketball     NaN  
11690  Basketball Women's Basketball  Bronze  
21295  Basketball Women's Basketball     NaN  
21296  Basketball Women's Basketball     NaN  
39283    Basketball Men's Basketball     NaN

In [6]:
df_OLY.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [7]:
df_OLY.loc[:,'Year'] = pd.to_datetime(df_OLY.loc[:,'Year'])
df_OLY.dtypes

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ID                 int64
Name              object
Sex               object
Age              float64
Height           float64
Weight           float64
Team              object
NOC               object
Games             object
Year      datetime64[ns]
Season            object
City              object
Sport             object
Event             object
Medal             object
dtype: object

In [8]:
regions = pd.read_csv('noc_regions.csv')
merged = pd.merge(df, regions, on='NOC', how='left')

In [9]:
merged.head()

ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal       region notes  
0       Basketball Men's Basketball   NaN        China   NaN  
1      Judo Men's Extra-Lightweight   NaN        China   NaN  
2           Football Men's Football   NaN      Denmark   NaN  
3       Tug-Of-War Men's Tug-Of-War  Gold      Denmark   NaN  
4  Speed Skating Women's 500 metres   NaN  Netherlands   NaN

In [10]:
goldMedals = merged[(merged.Medal == 'Gold')]
goldMedals.head()

ID                     Name Sex   Age  Height  Weight            Team  \
3    4     Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
42  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0         Finland   
44  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0         Finland   
48  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0         Finland   
60  20       Kjetil Andr Aamodt   M  20.0   176.0    85.0          Norway   

    NOC        Games  Year  Season         City          Sport  \
3   DEN  1900 Summer  1900  Summer        Paris     Tug-Of-War   
42  FIN  1948 Summer  1948  Summer       London     Gymnastics   
44  FIN  1948 Summer  1948  Summer       London     Gymnastics   
48  FIN  1948 Summer  1948  Summer       London     Gymnastics   
60  NOR  1992 Winter  1992  Winter  Albertville  Alpine Skiing   

                               Event Medal   region notes  
3        Tug-Of-War Men's Tug-Of-War  Gold  Denmark   NaN  
42  Gymnastics Men's Team All-Around  Gold  Finland   NaN  
44      Gymnastics Men's Horse Vault  Gold  Finland   NaN  
48  Gymnastics Men's Pommelled Horse  Gold  Finland   NaN  
60       Alpine Skiing Men's Super G  Gold   Norway   NaN

In [11]:
goldMedals.isnull().any()

ID        False
Name      False
Sex       False
Age        True
Height     True
Weight     True
Team      False
NOC       False
Games     False
Year      False
Season    False
City      False
Sport     False
Event     False
Medal     False
region     True
notes      True
dtype: bool

In [12]:
goldMedals = goldMedals[np.isfinite(goldMedals['Age'])]

In [13]:
plt.figure(figsize=(20, 10))
plt.tight_layout()
sns.countplot(goldMedals['Age'])
plt.title('Distribution of Gold Medals')

Text(0.5, 1.0, 'Distribution of Gold Medals')

In [14]:
goldMedals['ID'][goldMedals['Age'] > 50].count()

65

In [15]:
masterDisciplines = goldMedals['Sport'][goldMedals['Age'] > 50]

In [16]:
plt.figure(figsize=(20, 10))
plt.tight_layout()
sns.countplot(masterDisciplines)
plt.title('Gold Medals for Athletes Over 50')

Text(0.5, 1.0, 'Gold Medals for Athletes Over 50')

In [17]:
athlete = df[df['Season'] == 'Summer']
athlete

ID                                 Name Sex   Age  Height  Weight  \
0            1                            A Dijiang   M  24.0   180.0    80.0   
1            2                             A Lamusi   M  23.0   170.0    60.0   
2            3                  Gunnar Nielsen Aaby   M  24.0     NaN     NaN   
3            4                 Edgar Lindenau Aabye   M  34.0     NaN     NaN   
26           8   Cornelia "Cor" Aalten (-Strannood)   F  18.0   168.0     NaN   
27           8   Cornelia "Cor" Aalten (-Strannood)   F  18.0   168.0     NaN   
29          10       Einar Ferdinand "Einari" Aalto   M  26.0     NaN     NaN   
31          12                    Jyri Tapani Aalto   M  31.0   172.0    70.0   
32          13                   Minna Maarit Aalto   F  30.0   159.0    55.5   
33          13                   Minna Maarit Aalto   F  34.0   159.0    55.5   
35          15                 Arvo Ossian Aaltonen   M  22.0     NaN     NaN   
36          15                 Arvo Ossian Aaltonen   M  22.0     NaN     NaN   
37          15                 Arvo Ossian Aaltonen   M  30.0     NaN     NaN   
38          15                 Arvo Ossian Aaltonen   M  30.0     NaN     NaN   
39          15                 Arvo Ossian Aaltonen   M  34.0     NaN     NaN   
41          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
42          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
43          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
44          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
45          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
46          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
47          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
48          17              Paavo Johannes Aaltonen   M  28.0   175.0    64.0   
49          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
50          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
51          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
52          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
53          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
54          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
55          17              Paavo Johannes Aaltonen   M  32.0   175.0    64.0   
...        ...                                  ...  ..   ...     ...     ...   
271054  135547        Viktoriya Viktorovna Zyabkina   F  23.0   174.0    62.0   
271055  135547        Viktoriya Viktorovna Zyabkina   F  23.0   174.0    62.0   
271056  135548               Yury Ivanovich Zyabrev   M  25.0   174.0    62.0   
271075  135552      Jan (Johann-) Zybert (Siebert-)   M  20.0     NaN     NaN   
271076  135553  Galina Ivanovna Zybina (-Fyodorova)   F  21.0   168.0    80.0   
271077  135553  Galina Ivanovna Zybina (-Fyodorova)   F  21.0   168.0    80.0   
271078  135553  Galina Ivanovna Zybina (-Fyodorova)   F  25.0   168.0    80.0   
271079  135553  Galina Ivanovna Zybina (-Fyodorova)   F  29.0   168.0    80.0   
271080  135553  Galina Ivanovna Zybina (-Fyodorova)   F  33.0   168.0    80.0   
271081  135554                         Bogusaw Zych   M  28.0   182.0    82.0   
271082  135554                         Bogusaw Zych   M  28.0   182.0    82.0   
271083  135554                         Bogusaw Zych   M  36.0   182.0    82.0   
271084  135554                         Bogusaw Zych   M  36.0   182.0    82.0   
271085  135554                         Bogusaw Zych   M  36.0   182.0    82.0   
271087  135556           Bogusaw Stanisaw Zychowicz   M  19.0   189.0    80.0   
271088  135556           Bogusaw Stanisaw Zychowicz   M  19.0   189.0    80.0   
271089  135557                         Dominik ycki   M  34.0   192.0    95.0   
271090  135557                         Domi

In [18]:
medal = df[df['Medal'] == 'Gold']
medal

ID                                       Name Sex   Age  Height  \
3            4                       Edgar Lindenau Aabye   M  34.0     NaN   
42          17                    Paavo Johannes Aaltonen   M  28.0   175.0   
44          17                    Paavo Johannes Aaltonen   M  28.0   175.0   
48          17                    Paavo Johannes Aaltonen   M  28.0   175.0   
60          20                         Kjetil Andr Aamodt   M  20.0   176.0   
73          20                         Kjetil Andr Aamodt   M  30.0   176.0   
76          20                         Kjetil Andr Aamodt   M  30.0   176.0   
78          20                         Kjetil Andr Aamodt   M  34.0   176.0   
79          21                  Ragnhild Margrethe Aamodt   F  27.0   163.0   
113         40                            Roald Edgar Aas   M  31.0     NaN   
117         42                        Thomas Valentin Aas   M  25.0     NaN   
150         56                                 Ren Abadie   M  21.0     NaN   
172         72           Aleksey Aleksandrovich Abalmasov   M  28.0   180.0   
173         73                                  Luc Abalo   M  23.0   182.0   
174         73                                  Luc Abalo   M  27.0   182.0   
178         76                 Jouan Patrice Abanda Etong   M  22.0   185.0   
182         80                     Jos Mara Abarca Plotas   M  22.0   186.0   
188         85                    Alejandro Abascal Garca   M  28.0   181.0   
200         93                             Jol Marc Abati   M  38.0   190.0   
218        106                         Agostino Abbagnale   M  22.0   188.0   
219        106                         Agostino Abbagnale   M  29.0   188.0   
220        106                         Agostino Abbagnale   M  34.0   188.0   
221        107                          Carmine Abbagnale   M  22.0   182.0   
222        107                          Carmine Abbagnale   M  26.0   182.0   
226        108                         Giuseppe Abbagnale   M  25.0   187.0   
227        108                         Giuseppe Abbagnale   M  29.0   187.0   
283        150              Margaret Ives Abbott (-Dunne)   F  23.0     NaN   
524        297                          Lala Abdul Rashid   M  38.0   169.0   
550        316           Zagalav Abdulbekovich Abdulbekov   M  26.0   160.0   
587        337                 Mukhammad Kadyr Abdullayev   M  26.0   169.0   
...        ...                                        ...  ..   ...     ...   
270411  135262                               Angelo Zorzi   M  29.0     NaN   
270416  135263                             Cristian Zorzi   M  33.0   184.0   
270431  135269                                    Zou Kai   M  20.0   158.0   
270432  135269                                    Zou Kai   M  20.0   158.0   
270433  135269                                    Zou Kai   M  20.0   158.0   
270434  135269                                    Zou Kai   M  24.0   158.0   
270435  135269                                    Zou Kai   M  24.0   158.0   
270439  135270                                Zou Shiming   M  27.0   165.0   
270440  135270                                Zou Shiming   M  31.0   165.0   
270477  135288                              Zdravko Zovko   M  29.0   183.0   
270480  135290                    Vera Zozua (-Pyatnizin)   F  24.0   157.0   
270514  135294                                Vedran Zrni   M  24.0   188.0   
270552  135313                             Gyula Zsivtzky   M  31.0   190.0   
270588  135331                 Olha Valentynivna Zubareva   F  22.0   182.0   
270609  135343                 Aleksandr Yuryevich Zubkov   M  39.0   189.0   
270610  135343                 Aleksandr Yuryevich Zubkov   M  39.0   189.0   
270641  135355                  Sergey Mikhaylovich Zubov   M  21.0   186.0   
270676  135366                        Francesco Zucchetti   M  22.0     NaN   
270770  135414                               Andreas Zlow   M 

In [19]:
good = medal[medal['Year']==2000]
good

ID                                               Name Sex   Age  \
178         76                         Jouan Patrice Abanda Etong   M  22.0   
220        106                                 Agostino Abbagnale   M  34.0   
587        337                         Mukhammad Kadyr Abdullayev   M  26.0   
591        339                           Namiq Yadulla Abdullayev   M  29.0   
609        351                         Julius Shareef Abdur-Rahim   M  23.0   
720        411                                     Gezahgne Abera   M  22.0   
739        423                            Michael Brent Abernathy   M  22.0   
2184      1221             Taismary "Tai" Agero Leiva (-Botteghi)   F  23.0   
2689      1498                     Charles Benedict "Ben" Ainslie   M  23.0   
2696      1500                              Kurt Harold Ainsworth   M  22.0   
2756      1534                                       Brett Aitken   M  29.0   
4326      2469                                  Virgilijus Alekna   M  28.0   
4784      2735                     Sergey Gennadyevich Alifirenko   M  41.0   
4996      2850                            Kate Ruth "Katie" Allen   F  26.0   
5031      2863                                   Walter Ray Allen   M  25.0   
5318      3012                                   Nicolas Alnoudji   M  20.0   
5829      3281                               Simona Amnar (-Tabr)   F  20.0   
5830      3281                               Simona Amnar (-Tabr)   F  20.0   
5988      3363                                   Christie Ambrosi   F  23.0   
6613      3701                            Camilla Rseler Andersen   F  27.0   
6764      3793                   Andrea Arlene Anderson (-Bolder)   F  22.0   
7548      4207                                    Michel Andrieux   M  33.0   
7981      4418                                Alyson Regina Annan   F  27.0   
9405      5163                Jennifer Margaret "Jenny" Armstrong   F  30.0   
9722      5331            Samantha "Sam" Arsenault (-Livingstone)   F  18.0   
10292     5632                                      Brahim Asloum   M  21.0   
10732     5869                                 Louis Mark Attrill   M  25.0   
11269     6164                                       Nduka Awazie   M  19.0   
11534     6299                            Yelena Yuryevna Azarova   F  27.0   
11573     6305                           Filiberto Azcuy Aguilera   M  27.0   
...        ...                                                ...  ..   ...   
263786  132006                                      Xiao Hailiang   M  23.0   
263790  132009                                       Xiao Junfeng   M  21.0   
263856  132038                                         Xing Aowei   M  18.0   
263883  132045                                           Xiong Ni   M  26.0   
263884  132045                                           Xiong Ni   M  26.0   
264613  132416                                            Yan Sen   M  25.0   
264810  132523                                          Yang Ling   M  28.0   
264864  132547                                           Yang Wei   M  20.0   
264896  132552                                           Yang Xia   F  22.0   
265394  132807                   Vyacheslav Vladimirovich Yekimov   M  34.0   
265403  132811                          Yelena Borisovna Yelesina   F  30.0   
265561  132880                                    Hamza Yerlikaya   M  24.0   
265578  132885                          Oksana Ivanovna Yermakova   F  27.0   
266239  133203                        Ernest Wesley "Ernie" Young   M  31.0   
266319  133237                             Timothy R. "Tim" Young   M  26.0   
266658  133396                                           Yuan Hua   F  26.0   
266799  133479                                         Yun Mi-Jin   F  17.0   
266800  133479                                         Yun Mi-Jin   F  17.0   
267581  133869                  Yelena Mikhaylovna Zamolodchik

In [20]:
USA = medal[medal['NOC']=='USA']
USA

ID                                             Name Sex   Age  \
283        150                    Margaret Ives Abbott (-Dunne)   F  23.0   
609        351                       Julius Shareef Abdur-Rahim   M  23.0   
710        404                      Louis Grenville "Lou" Abell   M  15.0   
711        404                      Louis Grenville "Lou" Abell   M  19.0   
739        423                          Michael Brent Abernathy   M  22.0   
787        454                        Edgar Allen "Ed" Ablowich   M  19.0   
1239       699                  Charles Edwin "Charley" Ackerly   M  22.0   
1345       762                       Gordon Belgum "Gordy" Adam   M  21.0   
1472       818                               Harry Lester Adams   M  31.0   
1511       836                                      Platt Adams   M  27.0   
1727       962                           Charles "Chuck" Adkins   M  20.0   
1728       963                             Derrick Ralph Adkins   M  26.0   
1843      1017                           Nathan Ghar-Jun Adrian   M  19.0   
1844      1017                           Nathan Ghar-Jun Adrian   M  23.0   
1846      1017                           Nathan Ghar-Jun Adrian   M  23.0   
1849      1017                           Nathan Ghar-Jun Adrian   M  27.0   
1850      1017                           Nathan Ghar-Jun Adrian   M  27.0   
2031      1122                                Andre Kirk Agassi   M  26.0   
2410      1350                             George Lewis Ahlgren   M  19.0   
2466      1380  Christine M. "Crissy" Ahmann-Leighton (-Perham)   F  22.0   
2468      1380  Christine M. "Crissy" Ahmann-Leighton (-Perham)   F  22.0   
2579      1444                         Christian "Chris" Ahrens   M  28.0   
2696      1500                            Kurt Harold Ainsworth   M  22.0   
2911      1622                              Michelle Anne Akers   F  30.0   
3020      1682                                Morolake Akinosun   F  22.0   
4157      2386       Tenley Emma Albright (-Gardiner, -Blakely)   F  20.0   
4268      2440                   Frederick Pitt "Fred" Alderman   M  22.0   
4623      2636                      Stephen Todd "Steve" Alford   M  19.0   
4965      2832                     William Charles "Bill" Allen   M  24.0   
5031      2863                                 Walter Ray Allen   M  25.0   
...        ...                                              ...  ..   ...   
263025  131614                                    Kelsi Worrell   F  22.0   
263072  131638                        David James "Dave" Wottle   M  22.0   
263156  131676                                Heaton Luse Wrenn   M  20.0   
263175  131684                             Beals Coleman Wright   M  24.0   
263176  131684                             Beals Coleman Wright   M  24.0   
263201  131700                             Frank Seymour Wright   M  41.0   
263219  131711                       Lorenzo Christopher Wright   M  21.0   
263270  131740               Bernard Charles "Bernie" Wrightson   M  24.0   
263274  131743                                John Henry Writer   M  27.0   
263645  131929                            Frank Clifford Wykoff   M  18.0   
263646  131929                            Frank Clifford Wykoff   M  22.0   
263648  131929                            Frank Clifford Wykoff   M  26.0   
263656  131935                                      Peter Wylde   M  39.0   
264335  132284     Kristine Tsuya "Kristi" Yamaguchi (-Hedican)   F  20.0   
265587  132887                         John Lloyd "Jack" Yerman   M  21.0   
266075  133114                William Albert "Bill" Yorzyk, Jr.   M  23.0   
266196  133185                        Albert "Al" Young (Jung-)   M  26.0   
266222  133194                         Cyrus J. "Cy" Young, Jr.   M  23.0   
266224  133195                   Dannette Louise Young (-Stone)   F  23.0   
266230  133197                                 David King Young   M  21.0   
266238  

In [25]:
Summer = USA[USA['Season'] == 'Summer']
Summer

ID                                             Name Sex   Age  \
283        150                    Margaret Ives Abbott (-Dunne)   F  23.0   
609        351                       Julius Shareef Abdur-Rahim   M  23.0   
710        404                      Louis Grenville "Lou" Abell   M  15.0   
711        404                      Louis Grenville "Lou" Abell   M  19.0   
739        423                          Michael Brent Abernathy   M  22.0   
787        454                        Edgar Allen "Ed" Ablowich   M  19.0   
1239       699                  Charles Edwin "Charley" Ackerly   M  22.0   
1345       762                       Gordon Belgum "Gordy" Adam   M  21.0   
1472       818                               Harry Lester Adams   M  31.0   
1511       836                                      Platt Adams   M  27.0   
1727       962                           Charles "Chuck" Adkins   M  20.0   
1728       963                             Derrick Ralph Adkins   M  26.0   
1843      1017                           Nathan Ghar-Jun Adrian   M  19.0   
1844      1017                           Nathan Ghar-Jun Adrian   M  23.0   
1846      1017                           Nathan Ghar-Jun Adrian   M  23.0   
1849      1017                           Nathan Ghar-Jun Adrian   M  27.0   
1850      1017                           Nathan Ghar-Jun Adrian   M  27.0   
2031      1122                                Andre Kirk Agassi   M  26.0   
2410      1350                             George Lewis Ahlgren   M  19.0   
2466      1380  Christine M. "Crissy" Ahmann-Leighton (-Perham)   F  22.0   
2468      1380  Christine M. "Crissy" Ahmann-Leighton (-Perham)   F  22.0   
2579      1444                         Christian "Chris" Ahrens   M  28.0   
2696      1500                            Kurt Harold Ainsworth   M  22.0   
2911      1622                              Michelle Anne Akers   F  30.0   
3020      1682                                Morolake Akinosun   F  22.0   
4268      2440                   Frederick Pitt "Fred" Alderman   M  22.0   
4623      2636                      Stephen Todd "Steve" Alford   M  19.0   
4965      2832                     William Charles "Bill" Allen   M  24.0   
5031      2863                                 Walter Ray Allen   M  25.0   
5048      2874                                      Wyatt Allen   M  25.0   
...        ...                                              ...  ..   ...   
262956  131579                     William Tripp "Bill" Woolsey   M  17.0   
262982  131592                     James Theodore "Jim" Workman   M  20.0   
263025  131614                                    Kelsi Worrell   F  22.0   
263072  131638                        David James "Dave" Wottle   M  22.0   
263156  131676                                Heaton Luse Wrenn   M  20.0   
263175  131684                             Beals Coleman Wright   M  24.0   
263176  131684                             Beals Coleman Wright   M  24.0   
263201  131700                             Frank Seymour Wright   M  41.0   
263219  131711                       Lorenzo Christopher Wright   M  21.0   
263270  131740               Bernard Charles "Bernie" Wrightson   M  24.0   
263274  131743                                John Henry Writer   M  27.0   
263645  131929                            Frank Clifford Wykoff   M  18.0   
263646  131929                            Frank Clifford Wykoff   M  22.0   
263648  131929                            Frank Clifford Wykoff   M  26.0   
263656  131935                                      Peter Wylde   M  39.0   
265587  132887                         John Lloyd "Jack" Yerman   M  21.0   
266075  133114                William Albert "Bill" Yorzyk, Jr.   M  23.0   
266196  133185                        Albert "Al" Young (Jung-)   M  26.0   
266222  133194                         Cyrus J. "Cy" Young, Jr.   M  23.0   
266224  133195                   Dannette Louise Young (-Stone)   F  23.0   
266230  

In [26]:
gold_year = pd.DataFrame(Summer['Year'].value_counts())

In [27]:
gold_year.sort_index(inplace=True)

print(gold_year.to_string())

      Year
1896    11
1900    30
1904   128
1906    12
1908    34
1912    46
1920   111
1924    97
1928    47
1932    81
1936    51
1948    87
1952    83
1956    61
1960    81
1964    95
1968    99
1972    69
1976    70
1984   186
1988    87
1992    89
1996   159
2000   130
2004   117
2008   127
2012   145
2016   139


In [31]:
gold_year.sort_index()

Year
1896    11
1900    30
1904   128
1906    12
1908    34
1912    46
1920   111
1924    97
1928    47
1932    81
1936    51
1948    87
1952    83
1956    61
1960    81
1964    95
1968    99
1972    69
1976    70
1984   186
1988    87
1992    89
1996   159
2000   130
2004   117
2008   127
2012   145
2016   139

In [33]:
from statsmodels.tsa.ar_model import AR
from random import random
# contrived dataset
data = gold_year
# fit model
model = AR(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)


28    157.550515
dtype: float64


/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:531: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
